In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
from sklearn.utils.class_weight import compute_class_weight
import random
import seaborn as sns
use_cuda=torch.cuda.is_available()

print(use_cuda)

True


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pickle
import gzip
with gzip.open('/content/gdrive/MyDrive/APS360 Project DATA/MyDataFeatureFull.pkl', 'rb') as f: #\\MyDataFeatureFull
    loaded_data = pickle.load(f)

In [ ]:
class EEG_CNN(nn.Module)
    def __init__(self, name="EEG_CNN"):
        super(EEG_CNN,self).__init__()

        self.name=name

        self.conv1 = nn.Conv2d(1, 32, kernel_size = (3, 3), padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = (3, 3), padding=1)
        self.pool = nn.MaxPool2d(2,2)

        # Adjust dimensions for fully connected layers
        self.fc1 = nn.Linear(64*5*2, 10)  # self.fc1 = nn.Linear(64*5*2, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # output dimensions: [32, 11, 4]
        # print(x.size())
        x = self.pool(F.relu(self.conv2(x))) # output dimensions: [64, 5, 2]
        # print(x.size())
        x = x.view(-1, 64*5*2) # flatten layer x.view(-1, 64*5*2) (-1, 64,23,9) x.view(x.size(0), -1)
        # print(x.size())
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # print(x.size())
        return x

SyntaxError: ignored

In [ ]:
class ANN_MNISTClassifier(nn.Module):
    def __init__(self, name="EEG_CNN"):
        super(ANN_MNISTClassifier, self).__init__()

        self.name=name

        self.fc1 = nn.Linear(23*9, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, img):
        flattened = img.view(-1, 23*9)
        activation1 = F.relu(self.fc1(flattened))
        activation2 = F.relu(self.fc2(activation1))
        output = self.fc3(activation2)
        return output

In [ ]:
class EEG_CNN2(nn.Module):
    def __init__(self, name="EEG_CNN2"):
        super(EEG_CNN2,self).__init__()

        self.name=name

        self.conv1 = nn.Conv2d(1, 32, kernel_size = (3, 3), padding=(1))
        self.bn1 = nn.BatchNorm2d(32)  # Batch normalization after conv1
        self.conv2 = nn.Conv2d(32, 64, kernel_size = (3, 3), padding=(1))
        self.bn2 = nn.BatchNorm2d(64)  # Batch normalization after conv2
        self.conv3 = nn.Conv2d(64, 128, kernel_size = (3, 3), padding=(1))  # Additional convolutional layer
        self.bn3 = nn.BatchNorm2d(128)  # Batch normalization after conv3
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(0.3)  # Dropout for regularization

        # Adjust dimensions for fully connected layers
        # Update dimensions based on the output size from the conv layers
        self.fc1 = nn.Linear(128*2*1, 64)  # Adjusted based on conv3 and pooling layers
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Additional fully connected layer

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Add batch norm after conv1
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Add batch norm after conv2
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Add conv3 and batch norm

        x = x.view(-1, 128*2*1)  # Flatten layer, adjust based on conv3 and pooling layers
        x = self.dropout(F.relu(self.fc1(x)))  # Add dropout after activation
        x = self.dropout(F.relu(self.fc2(x)))  # Add dropout after activation

        x = self.fc3(x)  # Additional fully connected layer
        return x

In [ ]:
class EEG_CNN3(nn.Module):
    def __init__(self, name="EEG_CNN3"):
        super(EEG_CNN3,self).__init__()

        self.name=name

        self.conv1 = nn.Conv2d(1, 32, kernel_size = (3, 3), padding=(1))
        self.bn1 = nn.BatchNorm2d(32)  # Batch normalization after conv1
        self.conv2 = nn.Conv2d(32, 64, kernel_size = (3, 3), padding=(1))
        self.bn2 = nn.BatchNorm2d(64)  # Batch normalization after conv2
        self.conv3 = nn.Conv2d(64, 128, kernel_size = (3, 3), padding=(1))  # Additional convolutional layer
        self.bn3 = nn.BatchNorm2d(128)  # Batch normalization after conv3
        self.pool = nn.MaxPool2d(2,2)

        # Adjust dimensions for fully connected layers
        # Update dimensions based on the output size from the conv layers
        self.fc1 = nn.Linear(128*2*1, 64)  # Adjusted based on conv3 and pooling layers
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)  # Additional fully connected layer

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # Add batch norm after conv1
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  # Add batch norm after conv2
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Add conv3 and batch norm

        x = x.view(-1, 128*2*1)  # Flatten layer, adjust based on conv3 and pooling layers

        x = F.relu(self.fc1(x))  # Add dropout after activation
        x = F.relu(self.fc2(x))  # Add dropout after activation

        x = self.fc3(x)  # Additional fully connected layer
        return x

In [ ]:
torch.manual_seed(10)
def trim_channels(lst, max_channels=23):
    trimmed_lst = []
    for data, label in lst:
        # If data contains more than max_channels channels, trim it
        if len(data) > max_channels:
            data = data[:max_channels]
        if len(data) == max_channels:
            trimmed_lst.append((data, label))
    return trimmed_lst

loaded_data = trim_channels(loaded_data)

import numpy as np
from sklearn.model_selection import train_test_split
# Extract labels only for stratified splitting
labels = [label for data, label in loaded_data]
# Convert labels to binary (0 = False, 1 = True) for stratification
binary_labels = [1 if label else 0 for label in labels]
# Perform stratified splitting
train_data, temp_data = train_test_split(loaded_data, stratify=binary_labels, test_size=0.4)
# Now split the temporary set into equal halves of validation and test sets.
binary_labels_temp = [1 if label else 0 for data, label in temp_data]
val_data, test_data = train_test_split(temp_data, stratify=binary_labels_temp, test_size=0.5)

from sklearn.utils import resample
# separate the True and False samples in train_data
true_setT = [data for data in train_data if data[1] == True]
false_setT = [data for data in train_data if data[1] == False]
# oversample True class in train_data
true_setT_oversampled = resample(true_setT, replace=True, n_samples=len(false_setT), random_state=123)
# undersample False class in train_data
false_setT_undersampled = resample(false_setT, replace=False, n_samples=len(true_setT_oversampled), random_state=123)
# combine them to create a balanced train dataset
balanced_train = true_setT_oversampled + false_setT_undersampled
# shuffle the data
random.shuffle(balanced_train)
# separate the True and False samples in val_data
true_setV = [data for data in val_data if data[1] == True]
false_setV = [data for data in val_data if data[1] == False]
# oversample True class in val_data
true_setV_oversampled = resample(true_setV, replace=True, n_samples=len(false_setV), random_state=123)
# undersample False class in val_data
false_setV_undersampled = resample(false_setV, replace=False, n_samples=len(true_setV_oversampled), random_state=123)
# combine them to create a balanced validation dataset
balanced_val = true_setV_oversampled + false_setV_undersampled
# shuffle the data
random.shuffle(balanced_val)
print(len(false_setT_undersampled))
print(len(true_setT_oversampled))

# Normalizing the data
from sklearn.preprocessing import MinMaxScaler

def minmax_scale_data(data):
    """Scale the features in a dataset to a range between 0 and 1."""
    scaler = MinMaxScaler()
    return [(scaler.fit_transform(features), label) for features, label in data]

# Apply the MinMax scaling
train_dataN = minmax_scale_data(balanced_train)
val_dataN = minmax_scale_data(balanced_val)
test_dataN = minmax_scale_data(test_data)


def get_metrics(model, data, batch_size):
    torch.manual_seed(1)
    correct = 0
    total = 0
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    tn = 0  # True negatives
    counter = 0


    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=batch_size):
        imgs = imgs.unsqueeze(1)
        if use_cuda and torch.cuda.is_available():
            imgs = imgs.cuda()
            labels = labels.cuda()
            model = model.cuda()
        output = model(imgs.float())
        counter +=1

        pred = output.max(1, keepdim=True)[1]
        pred = pred.squeeze()
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]

        tp += ((pred == 1) & (labels == 1)).sum().item()
        fp += ((pred == 1) & (labels == 0)).sum().item()
        fn += ((pred == 0) & (labels == 1)).sum().item()
        tn += ((pred == 0) & (labels == 0)).sum().item()


    accuracy = correct / total
    precision = tp / (tp + fp) if tp + fp != 0 else 0.0
    recall = tp / (tp + fn) if tp + fn != 0 else 0.0
    f1 = 2*((recall*precision)/(recall+precision)) if recall + precision != 0 else 0.0
    confusion_matrix = torch.tensor([[tp, fp], [fn, tn]])

    return accuracy, recall, precision, f1, confusion_matrix


38910
38910


In [ ]:
def train(model, train_data, val_data, batch_size, num_epochs, learning_rate=0.01, MASTER_SAVE=True):
    torch.manual_seed(1)
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    criterion = nn.CrossEntropyLoss()  #criterion = nn.CrossEntropyLoss(weight=class_weights)

    epoch = 0
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    counter = 0
    epochs, iters, losses, train_acc, val_acc = [], [], [], [], []
    ct = 0
    n = 0
    model.to(device)


    # Current model's metrics - validation
    highest_accuracy = -1.0
    highest_precision = -1.0
    highest_recall = -1.0
    highest_f1 = -1.0


    best_paths = [' ',' ',' ',' ']

    for epoch in range(num_epochs):
        random.seed(1)
        random.shuffle(train_data)
        random.shuffle(val_data)
        tp = 0  # True positives
        fp = 0  # False positives
        fn = 0  # False negatives
        tn = 0  # True negatives

        train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
        for imgs, labels in iter(train_loader):
            imgs = imgs.unsqueeze(1)
            if use_cuda and torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()
            counter +=1
            labels = labels.long()  # Change labels to long
            if use_cuda and torch.cuda.is_available():
                model.cuda()
            else:
                device = torch.device("cpu")

            out = model(imgs.float())
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            iters.append(n)
            losses.append(float(loss)/batch_size)
            n += 1

            # print((out[0][0]))
            # print(len(out[0]))
            # pred = out.max(1, keepdim=True)[1]
            # # for i in range(len(pred)):
            # #     tp += ((pred[i][0] == 1) & (labels[i] == 1))
            # #     fp += ((pred[i][0] == 1) & (labels[i] == 0))
            # #     fn += ((pred[i][0] == 0) & (labels[i] == 1))
            # #     tn += ((pred[i][0] == 0) & (labels[i] == 0))
            # # print((pred == 1)[0])
            # pred = pred.squeeze()
            # # print(pred)
            # # print(1,(pred)[1])
            # # print(2,(labels)[1])
            # # print((pred)[0]==(labels)[0])
            # # print((pred[0] == 1) & (labels[0] == 1))
            # tp += ((pred == 1) & (labels == 1)).sum().item()
            # fp += ((pred == 1) & (labels == 0)).sum().item()
            # fn += ((pred == 0) & (labels == 1)).sum().item()
            # tn += ((pred == 0) & (labels == 0)).sum().item()
            # print(tn)
            # accuracyT, recallT, precisionT, f1T = get_metrics(model, train_data, batch_size=batch_size)
            # train_acc.append(accuracyT)
            # accuracyV, recallV, precisionV, f1V = get_metrics(model, val_data, batch_size=batch_size)
            # val_acc.append(accuracyV)
        print(epoch)
        epochs.append(epoch)
        epoch +=1
        accuracyT, recallT, precisionT, f1T, confusion_matrixT  = get_metrics(model, train_data, batch_size=batch_size)
        train_acc.append(accuracyT)
        accuracyV, recallV, precisionV, f1V, confusion_matrixV = get_metrics(model, val_data, batch_size=batch_size)
        val_acc.append(accuracyV)

        # Update models 'highest metrics'
        highest_accuracy = (highest_accuracy, accuracyV)[accuracyV > highest_accuracy]
        highest_precision = (highest_precision, precisionV)[precisionV > highest_precision]
        highest_recall = (highest_recall, recallV)[recallV > highest_recall]
        highest_f1 = (highest_f1, f1V)[f1V > highest_f1]

        do_save, reason = determine_save(accuracyV, recallV, precisionV, f1V,
                                         highest_accuracy, highest_precision,
                                         highest_recall, highest_f1)


        if(MASTER_SAVE and do_save):
          model_path = get_model_name(model.name, batch_size,learning_rate, accuracyV, recallV, precisionV, f1V, epoch, reason)
          best_paths[reason] = model_path
          save_model(model, model_path)

    print(n)
        # print(epoch)
    plt.title('Training Curve (' + str(model.name) + ')')
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()
    print(len(epochs))
    print(len(train_acc))
    plt.title('Training Curve (' + str(model.name) + ')')
    plt.plot(epochs, train_acc, label="Train")
    plt.plot(epochs, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Training Accuracy")
    plt.legend(loc='best')
    plt.show()

    # print(tp+fp+fn+tn)
    # confusion_matrix = torch.tensor([[tp, fp], [fn, tn]])
    plt.figure(figsize=(10, 7))
    sns.heatmap(confusion_matrixT.cpu(), annot=True, cmap='Blues')
    plt.xlabel('Predicted Labels')
    plt.ylabel('Actual Labels')
    plt.title('Confusion Matrix for Validation Data (' + str(model.name) + ')')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    print("Final Training Accuracy/recall/precision/f1:", accuracyT, recallT, precisionT, f1T)
    print("Final Validation Accuracy/recall/precision/f1:", accuracyV, recallV, precisionV, f1V)

    print("\nHighest of each metric:")
    print("USE THESE TO UPDATE get_save_metrics (copy paste)")
    print("save_accuracy = " + str(highest_accuracy))
    print("save_precision = " + str(highest_precision))
    print("save_recall = " + str(highest_recall))
    print("save_f1 = " + str(highest_f1))

    print("\nCorresponding Model Names: ")
    print("DOWNLOAD THESE -> BEST YOU HAVE SO FAR")
    print("Accuracy: " + str(best_paths[0]))
    print("Precision: " + str(best_paths[1]))
    print("Recall: " + str(best_paths[2]))
    print("F1: " + str(best_paths[3]))

In [ ]:
#MUST UPDATE THESE AFTER EACH TRAINING MODEL

# PASTE HERE
def get_save_metrics():

  save_recall = 0.9414803392444102
  save_accuracy = 0.9459907478797225
  save_precision = 0.9858392712185442
  save_f1 = 0.9451298319821407

  return save_accuracy, save_precision, save_recall, save_f1

def determine_save(accuracy, recall, precision, f1,
                                         highest_accuracy, highest_precision,
                                         highest_recall, highest_f1):
  save_accuracy, save_precision, save_recall, save_f1 = get_save_metrics()
  do_save = False

  save_accuracy = (save_accuracy, highest_accuracy)[highest_accuracy > save_accuracy]
  save_precision = (save_precision, highest_precision)[highest_precision > save_precision]
  save_recall = (save_recall, highest_recall)[highest_recall > save_recall]
  save_f1 = (save_f1, highest_f1)[highest_f1 > save_f1]

  if(f1 >= save_f1):
    do_save = True
    return do_save, 3

  if(precision >= save_precision):
    do_save = True
    return do_save, 1

  if(recall >= save_recall):
    do_save = True
    return do_save, 2

  if(accuracy >= save_accuracy):
    do_save = True
    return do_save, 0

  return False, ' '


def save_model(model, path):
  torch.save(model.state_dict(), path)


def get_model_name(name, batch_size, learning_rate, accuracy, recall, precision, f1, epoch, basis):

  accuracy = accuracy * 100
  recall = recall * 100
  precision = precision * 100
  f1 = f1 * 100

  # basis = basis[0].upper()

  word = ' '

  match basis:
    case 0:
      word = 'accuracy'
    case 1: word = 'precision'
    case 2: word = 'recall'
    case 3: word = 'f1'
    case _: word = 'error'

  path = "{reason}_{0}_bs{1}_lr{2}_a{3:.2f}_r{4:.2f}_p{5:.2f}_f{6:.2f}_epoch{7}".format(
                                  name,
                                  batch_size,
                                  learning_rate,
                                  accuracy,
                                  recall,
                                  precision,
                                  f1,
                                  epoch,
                                  reason=word)

  return path



In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
with_dropout = EEG_CNN2(name="dp_0.3")
if use_cuda and torch.cuda.is_available():
  with_dropout.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
batch_size=128
train(with_dropout, balanced_train, balanced_val, batch_size=batch_size, num_epochs = 25)

CUDA is not available.  Training on CPU ...
0


KeyboardInterrupt: ignored

In [ ]:
with_BN = EEG_CNN2(name="BN")
if use_cuda and torch.cuda.is_available():
  with_BN.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
batch_size=128
train(with_BN, balanced_train, balanced_val, batch_size=batch_size, num_epochs = 25)

In [ ]:
with_BN2 = EEG_CNN3(name="BN")
if use_cuda and torch.cuda.is_available():
  with_BN2.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
batch_size=256
train(with_BN2, balanced_train, balanced_val, batch_size=batch_size, num_epochs = 25)

In [ ]:
with_BN_dropout = EEG_CNN2(name="BN_dp_0.3")
if use_cuda and torch.cuda.is_available():
  with_BN_dropout.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
batch_size=128
train(with_BN_dropout, balanced_train, balanced_val, batch_size=batch_size, num_epochs = 25)

In [ ]:
default_CNN = EEG_CNN(name="default")
# if use_cuda and torch.cuda.is_available():
#   cnn.cuda()
torch.manual_seed(10)
if use_cuda and torch.cuda.is_available():
  default_CNN.cuda()
  print('CUDA is available!  Training on GPU ...')
else:
  print('CUDA is not available.  Training on CPU ...')
batch_size=128
train(default_CNN, balanced_train, balanced_val, batch_size=batch_size, num_epochs = 25)

In [ ]:
torch.manual_seed(10)
# model = EEG_CNN()
accuracy, recall, precision, f1, confusion_matrix = get_metrics(model,test_data, batch_size=128)
print(accuracy)
print(precision)
print(recall)
plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix.cpu(), annot=True, cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.title('Confusion Matrix for Validation Data')
plt.show()

In [ ]:
from sklearn import svm
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

concatenated_data = []
for i in range(len(loaded_data)):
    features = []
    for j in range(len(loaded_data[0][0])):
        for k in range(len(loaded_data[0][0][0])):
            features.append(loaded_data[i][0][j][k])
    concatenated_data.append((features, loaded_data[i][1]))

featuresSVM_train = []
labelsSVM_train = []
featuresSVM_test = []
labelsSVM_test= []
train_dataSVM, test_dataSVM = train_test_split(concatenated_data, stratify=binary_labels, test_size=0.3)

for i in range(len(train_dataSVM)):
    featuresSVM_train.append(train_dataSVM[i][0])
    labelsSVM_train.append(train_dataSVM[i][1])

for i in range(len(test_dataSVM)):
    featuresSVM_test.append(test_dataSVM[i][0])
    labelsSVM_test.append(test_dataSVM[i][1])

# Create an SVM classifier
clf = svm.SVC(kernel='poly') # Linear Kernel

# Define the hyperparameter space to search over
param_space = {
    'C': uniform(loc=0, scale=10),
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'rbf', 'poly']
}

# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Perform random search for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_space,
    n_iter=10,  # Number of parameter settings that are sampled
    cv=kfold,  # Cross-validation strategy
    scoring='precision',
    random_state=42,
    n_jobs=-1
)

random_search.fit(featuresSVM_train, labelsSVM_train)

# Retrieve the best hyperparameters and the corresponding model
best_params = random_search.best_params_
best_model = random_search.best_estimator_

# Evaluate the performance of the best model on the testing set
labels_pred = best_model.predict(featuresSVM_test)
accuracy = metrics.accuracy_score(labelsSVM_test, labels_pred)
recall = metrics.recall_score(labelsSVM_test, labels_pred)
# Print the best hyperparameters, precision, and accuracy
print(f'The best hyperparameters are {best_params}')
print(f'Precision on the testing dataset: {metrics.precision_score(labelsSVM_test, labels_pred):.4f}')
print(f'Accuracy on the testing dataset: {accuracy:.4f}')
print(f'Recall on the testing dataset: {recall:.4f}')


In [ ]:

from sklearn import svm
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

concatenated_data = []
for i in range(len(loaded_data)):
    features = []
    for j in range(len(loaded_data[0][0])):
        for k in range(len(loaded_data[0][0][0])):
            features.append(loaded_data[i][0][j][k])
    concatenated_data.append((features,loaded_data[i][1]))


featuresSVM_train = []
labelsSVM_train = []
featuresSVM_test = []
labelsSVM_test= []
train_dataSVM, test_dataSVM = train_test_split(concatenated_data, stratify=binary_labels, test_size=0.3)

for i in range(len(train_dataSVM)):
    featuresSVM_train.append(train_dataSVM[i][0])
    labelsSVM_train.append(train_dataSVM[i][1])

for i in range(len(test_dataSVM)):
    featuresSVM_test.append(test_dataSVM[i][0])
    labelsSVM_test.append(test_dataSVM[i][1])





#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(featuresSVM_train, labelsSVM_train)
#Predict the response for test dataset
labels_pred = clf.predict(featuresSVM_test)


from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy of tuned SVM:",metrics.accuracy_score(labelsSVM_test, labels_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision of tuned SVM:",metrics.precision_score(labelsSVM_test, labels_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall of tuned SVM:",metrics.recall_score(labelsSVM_test, labels_pred))

In [ ]:

from sklearn import svm
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval


concatenated_data = []
for i in range(len(loaded_data)):
    features = []
    for j in range(len(loaded_data[0][0])):
        for k in range(len(loaded_data[0][0][0])):
            features.append(loaded_data[i][0][j][k])
    concatenated_data.append((features,loaded_data[i][1]))


featuresSVM_train = []
labelsSVM_train = []
featuresSVM_test = []
labelsSVM_test= []
train_dataSVM, test_dataSVM = train_test_split(concatenated_data, stratify=binary_labels, test_size=0.3)

for i in range(len(train_dataSVM)):
    featuresSVM_train.append(train_dataSVM[i][0])
    labelsSVM_train.append(train_dataSVM[i][1])

for i in range(len(test_dataSVM)):
    featuresSVM_test.append(test_dataSVM[i][0])
    labelsSVM_test.append(test_dataSVM[i][1])





#Create a svm Classifier
model = SVC(C=0.07066, gamma='scale', kernel='linear')
#Train the model using the training sets
model.fit(featuresSVM_train, labelsSVM_train)
#Predict the response for test dataset
labels_pred = model.predict(featuresSVM_test)




]

from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy of tuned SVM:",metrics.accuracy_score(labelsSVM_test, labels_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision of tuned SVM:",metrics.precision_score(labelsSVM_test, labels_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall of tuned SVM:",metrics.recall_score(labelsSVM_test, labels_pred))

In [ ]:
from sklearn import svm
import pandas as pd
import numpy as np

# Standardize the data
from sklearn.preprocessing import StandardScaler

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

concatenated_data = []
for i in range(len(loaded_data)):
    features = []
    for j in range(len(loaded_data[0][0])):
        for k in range(len(loaded_data[0][0][0])):
            features.append(loaded_data[i][0][j][k])
    concatenated_data.append((features,loaded_data[i][1]))


featuresSVM_train = []
labelsSVM_train = []
featuresSVM_test = []
labelsSVM_test= []
train_dataSVM, test_dataSVM = train_test_split(concatenated_data, stratify=binary_labels, test_size=0.3)

for i in range(len(train_dataSVM)):
    featuresSVM_train.append(train_dataSVM[i][0])
    labelsSVM_train.append(train_dataSVM[i][1])

for i in range(len(test_dataSVM)):
    featuresSVM_test.append(test_dataSVM[i][0])
    labelsSVM_test.append(test_dataSVM[i][1])





#Create a svm Classifier
clff = SVC(C=0.07066, gamma='scale', kernel='linear')
#Train the model using the training sets
clff.fit(featuresSVM_train, labelsSVM_train)
#Predict the response for test dataset
labels_pred = clff.predict(featuresSVM_test)


from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy of tuned SVM:",metrics.accuracy_score(labelsSVM_test, labels_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision of tuned SVM:",metrics.precision_score(labelsSVM_test, labels_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall of tuned SVM:",metrics.recall_score(labelsSVM_test, labels_pred))

Accuracy of tuned SVM: 0.9831716191631291
Precision of tuned SVM: 0.0
Recall of tuned SVM: 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
def trim_channels(lst, max_channels=23):
    trimmed_lst = []
    for data, label in lst:
        # If data contains more than max_channels channels, trim it
        if len(data) > max_channels:
            data = data[:max_channels]
        if len(data) == max_channels:
            trimmed_lst.append((data, label))
    return trimmed_lst

loaded_data = trim_channels(loaded_data)
print(len(loaded_data))
# separate the True and False samples in train_data
true_setSVM = [data for data in loaded_data if data[1] == True]
false_setSVM = [data for data in loaded_data if data[1] == False]

# oversample True class in train_data
true_setSVM_oversampled = resample(true_setSVM, replace=True, n_samples=len(false_setSVM), random_state=123)
# undersample False class in train_data
false_setSVM_undersampled = resample(false_setSVM, replace=False, n_samples=len(true_setSVM_oversampled), random_state=123)

# combine them to create a balanced train dataset
balanced_SVM = true_setSVM_oversampled + false_setSVM_undersampled
# shuffle the data
random.shuffle(balanced_SVM)

print(len(balanced_SVM))

from sklearn import svm
loaded_data = balanced_SVM
concatenated_data = []
for i in range(len(loaded_data)):
    features = []
    for j in range(len(loaded_data[0][0])):
        for k in range(len(loaded_data[0][0][0])):
            features.append(loaded_data[i][0][j][k])
    concatenated_data.append((features,loaded_data[i][1]))


featuresSVM_train = []
labelsSVM_train = []
featuresSVM_test = []
labelsSVM_test= []

stratify_labels = [data[1] for data in concatenated_data] # remove

train_dataSVM, test_dataSVM = train_test_split(concatenated_data, stratify=stratify_labels, test_size=0.3) #(concatenated_data, stratify=binary_labels, test_size=0.3)
# train_dataSVM, test_dataSVM = balanced_train, balanced_val
for i in range(len(train_dataSVM)):
    featuresSVM_train.append(train_dataSVM[i][0])
    labelsSVM_train.append(train_dataSVM[i][1])

for i in range(len(test_dataSVM)):
    featuresSVM_test.append(test_dataSVM[i][0])
    labelsSVM_test.append(test_dataSVM[i][1])

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
# print(len(featuresSVM_train[0]))
# print(len(labelsSVM_train))
clf.fit(featuresSVM_train, labelsSVM_train)
#Predict the response for test dataset
labels_pred = clf.predict(featuresSVM_test)


from sklearn import metrics
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(labelsSVM_test, labels_pred))

# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(labelsSVM_test, labels_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(labelsSVM_test, labels_pred))

cm = metrics.confusion_matrix(labelsSVM_test, labels_pred)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()